In [1]:
import requests
from bs4 import BeautifulSoup

In [36]:
def scraping_describe(soup):
    contents = soup.select("div.articleMain > div > p")
    for content in contents:
        yield(content.text.strip())

In [88]:
import time
import re

def scraping_describe_all(session, url):
    print(url)
    response = session.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    pages = soup.select("p.counter > span")
    if len(pages) == 0:
        pages = 1
    else:
        txt = pages[0].text
        pages = int(re.search(r'/(\d+).*', txt).group(1))
    for i in range(1, pages+1):
        u = "{}&p={}".format(url, i)
        r = session.get(u)
        s = BeautifulSoup(r.text, "html.parser")
        for content in scraping_describe(s):
            yield(content)
        time.sleep(1)

In [89]:
def scraping_list(session, url):
    response = session.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    links = soup.select("#NSm > div > h2 > a")
    contents = []
    for link in links:
        content = "".join(scraping_describe_all(session, link["href"]))
        contents.append(content)
        break
    next_links = soup.select("#Sp1 > p > span > a")
    if len(next_links) != 0:
        l = next_links[-1]
        if l.text.startswith("次へ"):
            c = scraping_list(session, l["href"])
            contents = contents + c
    return contents

In [90]:
s = requests.Session()
contents = scraping_list(s, "https://news.yahoo.co.jp/search/?fr=news_sw&p=%E9%BA%A6%E8%8C%B6&ei=UTF-8&b=1")

https://headlines.yahoo.co.jp/article?a=20180322-00010002-vingtcinq-life
https://headlines.yahoo.co.jp/hl?a=20180222-00000004-maikirei-ent
https://headlines.yahoo.co.jp/article?a=20180119-00000016-pseven-soci
https://headlines.yahoo.co.jp/article?a=20171228-00005438-bunshun-ent
https://news.yahoo.co.jp/byline/ymjrky/20170723-00073620/


In [91]:
len(contents)

5

In [98]:
contents[4]

''